In [1]:
import pandas as pd
import numpy as np

import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

import sys

import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display


from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, learning_curve, validation_curve
from sklearn.neighbors import KNeighborsClassifier as kNN
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier


from IPython.display import Audio

from utils.audio_preprocess import *
from utils.SepTr import *
from utils.feature_exctraction import *

In [12]:
import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Activation, Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization, Input
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

#import warnings
#if not sys.warnoptions:
#    warnings.simplefilter("ignore")
#warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
# quante mad***e
from scikeras.wrappers import KerasClassifier

In [3]:
Crema = "CREMA-D/AudioWAV/"
crema_directory_list = os.listdir(Crema)

file_emotion = []
file_path = []

for file in crema_directory_list:

    file_path.append(Crema + file)

    part=file.split('_')
    if part[2] == 'SAD':
        file_emotion.append('sad')
    elif part[2] == 'ANG':
        file_emotion.append('angry')
    elif part[2] == 'DIS':
        file_emotion.append('disgust')
    elif part[2] == 'FEA':
        file_emotion.append('fear')
    elif part[2] == 'HAP':
        file_emotion.append('happy')
    elif part[2] == 'NEU':
        file_emotion.append('neutral')
    else:
        file_emotion.append('Unknown')
        
# dataframe 
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe Emotions/Path
path_df = pd.DataFrame(file_path, columns=['Path'])
Crema_df = pd.concat([emotion_df, path_df], axis=1)
Crema_df['Emotions'] = pd.Categorical(Crema_df['Emotions'])
Crema_df.head()

,Emotions,Path
0,angry,CREMA-D/AudioWAV/1022_ITS_ANG_XX.wav
1,angry,CREMA-D/AudioWAV/1037_ITS_ANG_XX.wav
2,neutral,CREMA-D/AudioWAV/1060_ITS_NEU_XX.wav
3,neutral,CREMA-D/AudioWAV/1075_ITS_NEU_XX.wav
4,disgust,CREMA-D/AudioWAV/1073_IOM_DIS_XX.wav


In [4]:
train, test = train_test_split(Crema_df, random_state=42, shuffle=True)

In [6]:
x_train, y_train = [], []
x_test, y_test = [], []

# TRAIN + Augmentation
for path, emotion in zip(train.Path, train.Emotions):
    feature = get_features(path, n_iter = 2, spect = True)
    for ele in feature:
        x_train.append(ele)
        y_train.append(emotion)
Train = pd.DataFrame(x_train)
Train['labels'] = y_train
#Train.to_csv('Data/train_synthless.csv', index=False)
Train.head()

# TEST
for path, emotion in zip(test.Path, test.Emotions):
    feature = get_features(path, synth = False, spect = True)
    for ele in feature:
        x_test.append(ele)
        y_test.append(emotion)
Test = pd.DataFrame(x_test)
Test['labels'] = y_test
#Test.to_csv('Data/test_synthless.csv', index=False)
Test.head()

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,labels
0,0.057724,0.021434,-328.098694,129.362656,1.603093,28.250675,-7.549809,-0.508716,1.917564,1.345103,...,8.036000e-09,7.320839e-09,6.836300e-09,6.490716e-09,6.242772e-09,6.069996e-09,5.948304e-09,5.871282e-09,5.824443e-09,angry
1,0.040913,0.014940,-422.495575,134.998413,20.309692,57.573116,-2.709049,24.978899,-15.263667,1.201231,...,7.019008e-11,6.662085e-11,6.391512e-11,6.179223e-11,6.017110e-11,5.901963e-11,5.817907e-11,5.764431e-11,5.732014e-11,sad
2,0.044956,0.018646,-368.973145,131.744995,8.143322,40.260132,-1.096721,6.814579,4.012649,0.106569,...,4.097419e-08,3.853548e-08,3.676714e-08,3.542807e-08,3.442318e-08,3.370419e-08,3.318075e-08,3.284814e-08,3.263901e-08,happy
3,0.062422,0.026883,-346.197937,138.361023,-13.182762,46.283558,-19.620443,12.619760,-10.991559,4.034292,...,6.982555e-08,6.486533e-08,6.133659e-08,5.869912e-08,5.672954e-08,5.531180e-08,5.427955e-08,5.361246e-08,5.319703e-08,happy
4,0.046660,0.027841,-432.640045,137.940247,20.167315,47.828022,-11.685402,30.969755,-15.280191,10.670103,...,4.504649e-09,4.032129e-09,3.708378e-09,3.474356e-09,3.303507e-09,3.181046e-09,3.093041e-09,3.035410e-09,3.000378e-09,sad


In [7]:
x_train = Train.iloc[: ,:-1].values
y_train = Train['labels'].values

x_test = Test.iloc[: ,:-1].values
y_test = Test['labels'].values

# one hot encoding
encoder = OneHotEncoder()

y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()

y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()

In [22]:
# scaling our data with sklearn's Standard scaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

# making our data compatible to model.
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((16743, 162, 1), (16743, 6), (1861, 162, 1), (1861, 6))

### CNN

In [23]:
def cnn_model(nfilters1=128, nfilters2=64, nfilters3=32, size_kernel=5, nstrides=1, size_pool=5):
    model=Sequential()
    model.add(Conv1D(filters=nfilters1, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu', input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(0.2))    
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(filters=nfilters2, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu'))
    model.add(Dropout(0.15))
    model.add(BatchNormalization())    
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))


    model.add(Conv1D(filters=nfilters3, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu'))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    #model.add(Conv1D(filters=nfilters4, kernel_size=size_kernel, strides=nstrides, padding='same', activation='relu'))
    #model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(units=16, activation='relu'))

    model.add(Dense(units=6, activation='softmax'))
    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

    #rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.30, verbose=0, patience=2, min_lr=0.0000001)

    #model.fit(x_train, y_train, batch_size=32, epochs=25, validation_data=(x_test, y_test), callbacks=[rlrp])    
    
    #model.summary()

    return model

In [24]:
cnn_model().fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

Epoch 1/20
524/524 [==============================] - 9s 16ms/step - loss: 1.5084 - accuracy: 0.3564 - val_loss: 1.5547 - val_accuracy: 0.3568
Epoch 2/20
524/524 [==============================] - 8s 16ms/step - loss: 1.4137 - accuracy: 0.4052 - val_loss: 1.4692 - val_accuracy: 0.3842
Epoch 3/20
524/524 [==============================] - 8s 16ms/step - loss: 1.3819 - accuracy: 0.4229 - val_loss: 1.5000 - val_accuracy: 0.4170
Epoch 4/20
524/524 [==============================] - 9s 16ms/step - loss: 1.3565 - accuracy: 0.4381 - val_loss: 1.4874 - val_accuracy: 0.3557
Epoch 5/20
524/524 [==============================] - 8s 16ms/step - loss: 1.3440 - accuracy: 0.4436 - val_loss: 1.4475 - val_accuracy: 0.4052
Epoch 6/20
524/524 [==============================] - 8s 16ms/step - loss: 1.3258 - accuracy: 0.4540 - val_loss: 1.3572 - val_accuracy: 0.4514
Epoch 7/20
524/524 [==============================] - 8s 15ms/step - loss: 1.3115 - accuracy: 0.4615 - val_loss: 1.4554 - val_accuracy: 0.4143